In [1]:
!pip install requests beautifulsoup4 kagglehub

# Captura de reseñas

In [2]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jaidalmotra/movies-review")

print("Path to dataset files:", path)

# List all files in the directory
path = os.path.join(path, os.listdir(path)[0])

# Create a dictionary to store DataFrames for each genre
genre_dfs = {}

# Read all CSV files
for file_name in os.listdir(path):
    df = pd.read_csv(os.path.join(path, file_name))

    # Extract the genre from the file name
    genre = file_name.split('.')[0]

    # Store the DataFrame in the dictionary
    genre_dfs[genre] = df

100%|██████████| 74.0k/74.0k [00:00<00:00, 18.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jaidalmotra/movies-review/versions/1


In [50]:
movies_df = pd.concat(genre_dfs.values(), ignore_index=True)
print("Total rows:", len(movies_df))
movies_df['id'] = movies_df['review_url'].str.extract(r'/title/(tt\d+)/')
movies_df = movies_df.drop_duplicates(subset='id').set_index('id')
# movies_df[movies_df['id'] == 'tt0034583']
print("Unique rows:", len(movies_df))

Total rows: 1700
Unique rows: 1150


In [51]:
movies_df.iloc[0]

,tt0114369
name,Se7en
year,1995
movie_rated,R
run_length,2h 7min
genres,Crime; Drama; Mystery;
release_date,22 September 1995 (USA)
rating,8.6
num_raters,1390231
num_reviews,1460
review_url,https://www.imdb.com/title/tt0114369/reviews/_...


In [52]:
movies_df.to_csv('movies.csv')

In [67]:
import requests
from bs4 import BeautifulSoup

def extract_imdb_reviews(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all review containers
        review_containers = soup.find_all('div', class_='review-container')

        reviews = []
        for container in review_containers:
            # Extract the review text
            review_text = container.find('div', class_='text show-more__control').text.strip()
            reviews.append(review_text)

        return reviews
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

reviews_data = []

# URL of the IMDB reviews page
for i, (movie_id, movie) in enumerate(movies_df.iterrows()):
  print("\rMovie count: ", i+1, end="")
  movie_name = movie['name']
  movie_url = movie['review_url']
  reviews = extract_imdb_reviews(movie_url)
  for review in reviews:
    reviews_data.append({'id': movie_id, 'name': movie_name, 'review': review})

# Convert the list of dictionaries into a DataFrame
reviews_df = pd.DataFrame(reviews_data)

# Set 'id' as the index
reviews_df.set_index('id', inplace=True)

reviews_df.head()

Movie count:  1150

,name,review
id,,
tt0114369,Se7en,Detective Lt. William Somerset (Morgan Freeman...
tt0114369,Se7en,It is a rarity for a film to be completely uns...
tt0114369,Se7en,"The movie, ""Se7en"", starring Brad Pitt, Morgan..."
tt0114369,Se7en,After his calamitous experience working on his...
tt0114369,Se7en,"From David Fincher, Se7en is a compelling and ..."


In [70]:
reviews_df.to_csv('reviews.csv')